In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)
# os.environ['CUDA_VISIBLE_DEVICES'] = "7"
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = "false"

slurm0-gpu1nodeset-0
/home/akarsh_sakana_ai/spaghetti/src
/home/akarsh_sakana_ai/.conda/envs/spaghetti/bin/python


In [2]:
import os, sys, glob, pickle, copy, time
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = (15, 5)
import seaborn as sns
sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [3]:
import jax
import jax.numpy as jnp
from jax.random import split

import flax
import flax.linen as nn
from flax.training.train_state import TrainState

import optax

In [4]:
from clip import CLIP
import cppn

In [5]:
clip = CLIP()

2025-01-13 21:05:35.913625: W external/xla/xla/service/gpu/nvptx_compiler.cc:836] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version (12.6.68). Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [6]:
imgs = []
img = plt.imread('../cat.jpg')/255.
img = jax.image.resize(img, (224, 224, 3), method='bilinear')
imgs.append(img)
img = plt.imread('../dog.jpg')/255.
img = jax.image.resize(img, (224, 224, 3), method='bilinear')
imgs.append(img)
imgs = jnp.stack(imgs)

z_txt = clip.embed_txt(['a cat', 'a dog', 'a bird'])
z_img = jax.vmap(clip.embed_img)(imgs)

Could not load symbol cuFuncGetName. Error: /lib/x86_64-linux-gnu/libcuda.so.1: undefined symbol: cuFuncGetName


In [7]:
scores = z_img @ z_txt.T
scores.shape

(2, 3)

In [8]:
scores

Array([[0.27269477, 0.22626628, 0.20600499],
       [0.18915439, 0.246398  , 0.1868664 ]], dtype=float32)

In [9]:
jax.nn.softmax(scores*100, axis=-1)

Array([[0.98921764, 0.0095264 , 0.00125601],
       [0.0032464 , 0.9941712 , 0.00258247]], dtype=float32)

In [10]:
@jax.jit
def loss_fn(img):
    z_img = clip.embed_img(img)
    return z_img.mean()
    # return img.mean()

grad_fn = jax.jit(jax.value_and_grad(loss_fn))
# grad_fn(img)

for i in tqdm(range(1000)):
    loss_fn(imgs[0])
    # grad_fn(imgs[0])

  0%|          | 0/1000 [00:00<?, ?it/s]